In [2]:
from sympy import *
from itertools import combinations

In [3]:
n = 4
s = 3
t = 3

p = [[None for _ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        p[i][j] = Symbol(f"p{i}{j}")

In [4]:
formula = And(False)

def comp_conn(s):
    formula = And(True)
    for i in s:
        for j in s:
            formula = And(formula, p[i][j])
    print(formula)
    return formula

def comp_disc(s):
    formula = And(True)
    for i in s:
        for j in s:
            formula = And(formula, Not(p[i][j]))
    print(formula)
    return formula

for c in combinations(list(range(n)), s):
    formula = Or(formula, comp_conn(c))
    
for c in combinations(list(range(n)), t):
    formula = Or(formula, comp_disc(c))

p00 & p01 & p02 & p10 & p11 & p12 & p20 & p21 & p22
p00 & p01 & p03 & p10 & p11 & p13 & p30 & p31 & p33
p00 & p02 & p03 & p20 & p22 & p23 & p30 & p32 & p33
p11 & p12 & p13 & p21 & p22 & p23 & p31 & p32 & p33
~p00 & ~p01 & ~p02 & ~p10 & ~p11 & ~p12 & ~p20 & ~p21 & ~p22
~p00 & ~p01 & ~p03 & ~p10 & ~p11 & ~p13 & ~p30 & ~p31 & ~p33
~p00 & ~p02 & ~p03 & ~p20 & ~p22 & ~p23 & ~p30 & ~p32 & ~p33
~p11 & ~p12 & ~p13 & ~p21 & ~p22 & ~p23 & ~p31 & ~p32 & ~p33


In [5]:
formula_cnf = to_cnf(Not(formula))
print(formula_cnf)

(p00 | p01 | p02 | p10 | p11 | p12 | p20 | p21 | p22) & (p00 | p01 | p03 | p10 | p11 | p13 | p30 | p31 | p33) & (p00 | p02 | p03 | p20 | p22 | p23 | p30 | p32 | p33) & (p11 | p12 | p13 | p21 | p22 | p23 | p31 | p32 | p33) & (~p00 | ~p01 | ~p02 | ~p10 | ~p11 | ~p12 | ~p20 | ~p21 | ~p22) & (~p00 | ~p01 | ~p03 | ~p10 | ~p11 | ~p13 | ~p30 | ~p31 | ~p33) & (~p00 | ~p02 | ~p03 | ~p20 | ~p22 | ~p23 | ~p30 | ~p32 | ~p33) & (~p11 | ~p12 | ~p13 | ~p21 | ~p22 | ~p23 | ~p31 | ~p32 | ~p33)


In [6]:
from pysat.solvers import Glucose4

def check_n(n, s, t):
    def combs(k, ss=list(range(1, n+1))):
        return combinations(ss, k)

    NUMS = {tuple(p): e + 1 for e, p in enumerate(combs(2))}
    def get_number(c):
        return NUMS[tuple(c)]


    g = Glucose4()

    for c in combs(s):
        clause = [get_number(t) for t in combs(2, c)]
        g.add_clause(clause)

    for c in combs(t):
        clause = [-get_number(t) for t in combs(2, c)]
        g.add_clause(clause)
    
    return g.solve()

In [7]:
from tqdm import tqdm

for s in tqdm(range(1, 8)):
    for t in tqdm(range(1, 8)):
        print(s, t)

        n = 1
        while check_n(n, s, t):
            n += 1
        print(s, t, n)

 57%|█████▋    | 4/7 [00:00<00:00, 36.50it/s]


1 1
1 1 1
1 2
1 2 1
1 3
1 3 1
1 4
1 4 1
1 5
1 5 1
1 6
1 6 1
1 7
1 7 1
2 1
2 1 1
2 2
2 2 2
2 3
2 3 3
2 4
2 4 4
2 5
2 5 5
2 6
2 6 6
2 7
2 7 7
3 1
3 1 1
3 2
3 2 3
3 3
3 3 6
3 4
3 4 9
3 5


 57%|█████▋    | 4/7 [49:56<00:00, 36.50it/s]


error: Caught keyboard interrupt